<center> <h1> Heterogeneous Computing for AI </h1> </center>

<center> <h2> Lecture 03 -: Exercises </h2> </center>

<center> <h4> Raghava Mukkamala (rrm.digi@cbs.dk)</h4> </center>

Instructions

Please use Python 3 for working on the following questions.


## Exercise 1

Please look at the locking-bug.py file (code available in the following cell as well) to answer this question. 

The code in this question is similar to the one discussed in class (**Example 2: Bank account with Mutex lock**). 
The only differences are that we have an additional decrease function which decreases the amount instead of increasing it.

We use two locks to ensure we resolve the race condition but unfortunately there is an issue with the code provided.
The race condition still persists. 

Please run the code to see the output.

In [ ]:
# locking-bug.py

'''
Standard example to display race condition & locking solution
'''

from threading import Thread, Lock
import time


# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val+= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value)
    lock.release()


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val -= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value, '\n')
    lock.release()



In [ ]:
# Test code for locking-bug.py

for i in range(10):

    shared_value = 0

    lock1 = Lock()
    lock2 = Lock()


    t1 = Thread(target= increase, args= (20,lock1))
    t2 = Thread(target= decrease, args= (10,lock2))

    # Start Threads
    t1.start()
    t2.start()

    # Join the threads so we wait for them to complete
    t1.join()
    t2.join()

    print("Final value of shared_value is: ", shared_value, '\n')
    print('**************************************')

# We expect the solution to be 10 but we either get -10 or 20.


Please fix the above code so that the issue disappears and explain why the issue was caused in the first place.

### Your solution

In [ ]:
# locking-bug.py

'''
Standard example to display race condition & locking solution
'''

from threading import Thread, Lock
import time


# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val+= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value)
    lock.release()


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()

    # Copy over the shared_value to a local value
    local_val = shared_value
    # Increase local_val by the amount specified
    local_val -= amount

    # Let's make the thread sleep for half a second
    time.sleep(0.5)

    # Update shared_value 
    shared_value = local_val

    print("Value of shared_value is: ", shared_value, '\n')
    lock.release()


In [ ]:
# Test code for locking-bug.py

for i in range(10):

    shared_value = 0

    lock = Lock()
#    lock2 = Lock()


    t1 = Thread(target= increase, args= (20,lock))
    t2 = Thread(target= decrease, args= (10,lock))

    # Start Threads
    t1.start()
    t2.start()

    # Join the threads so we wait for them to complete
    t1.join()
    t2.join()

    print("Final value of shared_value is: ", shared_value, '\n')
    print('**************************************')

# We expect the solution to be 10 but we either get -10 or 20.
# Explaination:
# In the original code it created two different locks.
# Which means they will not block each other from executing.
# Simply create one Lock() object and pass the same object to both threads

## Exercise 2

In [ ]:
'''
race condition & locking 
'''
from threading import Thread, Lock
import time


# Global Variables
NUM_TRANSACTIONS = 10

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    for i in range(NUM_TRANSACTIONS):
        read_val = shared_value
        # an additional 0.1 seconds simulates how normal processing times take
        # when work is done on a read_value 
        time.sleep(0.1)
        shared_value = read_val + amount
        print("In increase, share_value is: ", shared_value)


def decrease(amount: int) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    for i in range(NUM_TRANSACTIONS):
        read_val = shared_value
        time.sleep(0.1)
        shared_value = read_val - amount
        print("In decrease, share_value is: ", shared_value)


In [ ]:
shared_value = 0

t1 = Thread(target= increase, args= (20,))
t2 = Thread(target= decrease, args= (10,))

# Start Threads
t2.start()
t1.start()

# Join the threads so we wait for them to complete
t1.join()
t2.join()

print("Final value of shared_value is: ", shared_value)



### <font color='red'> We expect the solution to be 100, but we either get different values based on how, threads interleave between critical section </font>


### Exercise 2A: using Coarse-grained locking

Solve the race condition 
In this question, you are allowed to solve the race condition by locking such 
that a single thread can acquire the lock such that all the incraeses can be done
with the lock is held. 

### Your Solution

In [ ]:


'''
race condition & locking 
'''
from threading import Thread, Lock
import time


# Global Variables
NUM_TRANSACTIONS = 10

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()
    for i in range(NUM_TRANSACTIONS):
        
        read_val = shared_value
        # an additional 0.1 seconds simulates how normal processing times take
        # when work is done on a read_value 
        time.sleep(0.1)
        shared_value = read_val + amount
        print("In increase, share_value is: ", shared_value)
        
    lock.release()


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()
    for i in range(NUM_TRANSACTIONS):
        
        read_val = shared_value
        time.sleep(0.1)
        shared_value = read_val - amount
        print("In decrease, share_value is: ", shared_value)
        
    lock.release()



In [ ]:
start_time = time.time()
shared_value = 0
lock = Lock()

t1 = Thread(target= increase, args= (20, lock))
t2 = Thread(target= decrease, args= (10, lock))

# Start Threads
t2.start()
t1.start()

# Join the threads so we wait for them to complete
t1.join()
t2.join()

print("Final value of shared_value is: ", shared_value)
end_time = time.time()
print(f"Total time: {round(end_time - start_time, 3)}seconds")


### Exercise 2B: using Fine-grained locking 

In this question, use the locking mechanism in such a way that the lock is released 
as soon as a single increase or decrease occurs. 
Previously, a thread could lock until all increases have been performed.
Here, that is not allowed.

### Your Solution

In [ ]:


'''
race condition & locking 
'''
from threading import Thread, Lock
import time


# Global Variables
NUM_TRANSACTIONS = 10

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    for i in range(NUM_TRANSACTIONS):
        lock.acquire()
        read_val = shared_value
        # an additional 0.1 seconds simulates how normal processing times take
        # when work is done on a read_value 
        time.sleep(0.1)
        shared_value = read_val + amount
        print("In increase, share_value is: ", shared_value)
        lock.release()
        


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    for i in range(NUM_TRANSACTIONS):
        lock.acquire()
        read_val = shared_value
        time.sleep(0.1)
        shared_value = read_val - amount
        print("In decrease, share_value is: ", shared_value)
        lock.release()
        



In [ ]:
start_time = time.time()
shared_value = 0
lock = Lock()

t1 = Thread(target= increase, args= (20, lock))
t2 = Thread(target= decrease, args= (10, lock))

# Start Threads
t2.start()
t1.start()

# Join the threads so we wait for them to complete
t1.join()
t2.join()

print("Final value of shared_value is: ", shared_value)
end_time = time.time()
print(f"Total time: {round(end_time - start_time, 3)}seconds")

### Exercise 2C: Time your solutions 

Compare the time the differences between the two different locking strategies. 

### Your Solution

<p>In the first implementation -> Total time: 2.186seconds</p>

<p>In the second implementation -> Total time: 2.186seconds</p>

## Exercise 3: Putting Constraints

We are going to put an additional constraint that the shared_value 
can never be below 0, in the transactions from exercise no:2. 
This constraint reflects real-world requiremetns like
a bank account never going below 0.


### Exercise 3A: using Coarse-grained locking
Using a coarse locking strategy, implement the constraint above.


### Your Solution

In [2]:


'''
race condition & locking 
'''
from threading import Thread, Lock
import time


# Global Variables
NUM_TRANSACTIONS = 10

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    lock.acquire()
    for i in range(NUM_TRANSACTIONS):
        
        read_val = shared_value
        # an additional 0.1 seconds simulates how normal processing times take
        # when work is done on a read_value 
        time.sleep(0.1)
        shared_value = read_val + amount
        print("In increase, share_value is: ", shared_value)
        
    lock.release()


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    while shared_value <= 0:
        print("shared_value <= 0:", shared_value)
        time.sleep(0.1)
    
    lock.acquire()
    for i in range(NUM_TRANSACTIONS):
        
        read_val = shared_value
        time.sleep(0.1)
        shared_value = read_val - amount
        print("In decrease, share_value is: ", shared_value)
        
    lock.release()



In [3]:
start_time = time.time()
shared_value = 0
lock = Lock()

t1 = Thread(target= increase, args= (20, lock))
t2 = Thread(target= decrease, args= (10, lock))

# Start Threads
t2.start()
t1.start()

# Join the threads so we wait for them to complete
t1.join()
t2.join()

print("Final value of shared_value is: ", shared_value)
end_time = time.time()
print(f"Total time: {round(end_time - start_time, 3)}seconds")

shared_value <= 0: 0
In increase, share_value is:  20
In increase, share_value is:  40
In increase, share_value is:  60
In increase, share_value is:  80
In increase, share_value is:  100
In increase, share_value is:  120
In increase, share_value is:  140
In increase, share_value is:  160
In increase, share_value is:  180
In increase, share_value is:  200
In decrease, share_value is:  190
In decrease, share_value is:  180
In decrease, share_value is:  170
In decrease, share_value is:  160
In decrease, share_value is:  150
In decrease, share_value is:  140
In decrease, share_value is:  130
In decrease, share_value is:  120
In decrease, share_value is:  110
In decrease, share_value is:  100
Final value of shared_value is:  100
Total time: 2.221seconds


### Exercise 3B: using Fine-grained locking
Using a fine-grained locking strategy, implement the constraint above.


### Your Solution

In [4]:



'''
race condition & locking 
'''
from threading import Thread, Lock
import time


# Global Variables
NUM_TRANSACTIONS = 10

# We will be using a global variable as a shared resource among the threads
shared_value = 0

def increase(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value

    for i in range(NUM_TRANSACTIONS):
        lock.acquire()
        read_val = shared_value
        # an additional 0.1 seconds simulates how normal processing times take
        # when work is done on a read_value 
        time.sleep(0.1)
        shared_value = read_val + amount
        print("In increase, share_value is: ", shared_value)
        lock.release()
        


def decrease(amount: int, lock: Lock) -> None:
    # allows us to use the global value instead of local (function-limited) variable 
    global shared_value
    
    while shared_value <= 0:
        print("shared_value <= 0:", shared_value)
        time.sleep(0.1)

    for i in range(NUM_TRANSACTIONS):
        lock.acquire()
        read_val = shared_value
        time.sleep(0.1)
        shared_value = read_val - amount
        print("In decrease, share_value is: ", shared_value)
        lock.release()
        




In [5]:
start_time = time.time()
shared_value = 0
lock = Lock()

t1 = Thread(target= increase, args= (20, lock))
t2 = Thread(target= decrease, args= (10, lock))

# Start Threads
t2.start()
t1.start()

# Join the threads so we wait for them to complete
t1.join()
t2.join()

print("Final value of shared_value is: ", shared_value)
end_time = time.time()
print(f"Total time: {round(end_time - start_time, 3)}seconds")

shared_value <= 0: 0
shared_value <= 0:In increase, share_value is:  20
 0
In increase, share_value is:  40
In increase, share_value is:  60
In increase, share_value is:  80
In increase, share_value is:  100
In increase, share_value is:  120
In increase, share_value is:  140
In increase, share_value is:  160
In increase, share_value is:  180
In increase, share_value is:  200
In decrease, share_value is:  190
In decrease, share_value is:  180
In decrease, share_value is:  170
In decrease, share_value is:  160
In decrease, share_value is:  150
In decrease, share_value is:  140
In decrease, share_value is:  130
In decrease, share_value is:  120
In decrease, share_value is:  110
In decrease, share_value is:  100
Final value of shared_value is:  100
Total time: 2.213seconds


### Exercise 3C: Time your solutions 

Compare the time the differences between the two different locking strategies. 

### Your Solution

In [ ]:
Total time: 2.197seconds

In [ ]:
Total time: 2.179seconds

## Exercise 4: using Queues

This question realtes to using a queue for passing data between threads.
We will be looking into the producer/consumer model for this question.

In this question, we will be using 5 threads ("the producers") to download contents from 
5 different websites (as we did in exercises last week) and we will put 
the responses from the downloaded websites into a queue.
We will then use a different set of 5 threads ("the consumers") to process these
responses by pulling items from the queue.

Please find the file "queue_template.py" and use the structure found to answer this question.
The file provides a general structure with TODOs to work with in this question.

### Your Solution

In [1]:
import threading
import time
import queue
import requests

# Create queue 
items = queue.Queue()

def producer(url):
    print("Inside Producer",threading.current_thread().name, "working on: ",url)

    # TODO : download the contents of provided url and place in "items" queue
    with requests.get(url) as response:
        items.put((url, response))
        
    time.sleep(0.1)
    print("Inside Producer",threading.current_thread().name,
          ' done adding content to queue!')

def consumer():
    print("Inside Consumer", threading.current_thread().name)
    
    # TODO : retrieve contents from items and compute and then output len of contents
    url, resp = items.get()
    
    # For example:
    # print("For url: ", url, "Content length is: ",len(resp.content))
    print(threading.current_thread().name, ": consumer. For url: ",
          url, "Content length is: ",
          len(resp.content))
    time.sleep(0.1)
    items.task_done()


if __name__ == '__main__':


    urls = ["https://twitter.com",
            "https://facebook.com", 
            "https://www.linkedin.com", 
            "https://www.wikipedia.org/", 
            "https://github.com/"]


    # TODO: Initialise consumers first using threads 
    #       Use 5 threads in total, one for each url in urls
    consumers = [threading.Thread(target = consumer) for i in range(5)]

    for con in consumers:
        con.start()
        
    # TODO: Initialise producers to download the websites using threads
    #       Use 5 threads in total 
    producers = [threading.Thread(target = producer, args = (url,)) for url in urls]

    for prod in producers:
        prod.start() 


    # TODO: Figure out the order of joining consumer and producer threads
    for prod in producers:
        prod.join()
        
    
    # the following code items.join() blocks the main thread
    # until all items in the queue have been completed
    items.join()

    for con in consumers:
        con.join()


Inside Consumer Thread-5
Inside Consumer Thread-6
Inside Consumer Thread-7
Inside Consumer Thread-8
Inside Consumer Thread-9
Inside Producer Thread-10 working on:  https://twitter.com
Inside Producer Thread-11 Inside Producer Thread-12 working on:  https://facebook.com
Inside Producer Thread-13 working on: Inside Producer Thread-14  working on:  https://www.linkedin.com
working on:  https://github.com/
https://www.wikipedia.org/
Thread-5 : consumer. For url:  https://www.wikipedia.org/ Content length is:  75189
Inside Producer Thread-13  done adding content to queue!
Thread-6 : consumer. For url:  https://github.com/ Content length is:  309887
Thread-7 : consumer. For url:  https://twitter.com Content length is:  125434
Inside Producer Thread-14  done adding content to queue!
Inside Producer Thread-10  done adding content to queue!
Thread-8 : consumer. For url:  https://facebook.com Content length is:  64559
Thread-9 : consumer. For url:  https://www.linkedin.com Content length is:  12

## [Optional] Exercise 5: 

Implement the file downloader example in exercise 4 using one of the advanced threading primitives, e.g., semaphores/events/conditional variables.